In [ ]:
"""

當用戶想要重新觀看歡迎訊息時，可輸入特定文字，Server會重新寄發歡迎訊息

當用戶發送指定文字消息時，會發送相應的圖片消息給用戶

"""

In [1]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/food" , static_folder = "../food/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'
server_url

'0da3c1a2.ngrok.io'

In [2]:
token = secretFileContentJson.get("channel_access_token")
print(token)

5RNqdlI7TNU2iXy8VLLhuYY6XEqcLzzCAV2I4f6UAZ4s/l8jDBMvjQkzxvlVOLLqL8IRtN/3YaxjcXkZJMBSbqt82yGSJjD885hForRc7zAF298FMHmxQ+EixfVg6l+cd55SEQSj9U0ZhEwDPsfiJFGUYhWQfeY8sLGRXgo3xvw=


In [3]:
# MySQL套件
import pymysql

# 將QRcode上傳MySQL資料庫
conn = pymysql.connect(
    host = "uuboyscymysql.clrhltpp3icl.ap-northeast-1.rds.amazonaws.com",
    port = int(3306),user = "food",
    password = "food",
    db = "food",
    charset='utf8', 
    cursorclass=pymysql.cursors.DictCursor 
    )
cursor = conn.cursor()
print('Connected successfully!')

Connected successfully!


In [4]:
'''

製作文字與圖片發送消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 設定消息素材

'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
TextSendMessage(text="歡迎光臨無人超市"),
    TextSendMessage(text="更多資訊請點選下列選單")
]

'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息
4. 製作該用戶的QRcode
5. 將QRcode同時保存於資料庫及'../food'
6. 同時將userID上傳至資料庫

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests

# 製作QRcode的程式
import create_qrcode


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("../foodusers.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
    print()
        
        
    # 將菜單綁定在用戶身上
    linkRichMenuId=secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
    
    # 製作該使用者QRcode
    create_qrcode.create_qrcode(user_profile.user_id, user_profile.user_id)
    
    # 將使用者userID, name, qrcode 上傳至MySQL資料庫
    f = open('../food/%s.jpg'%(user_profile.user_id), 'rb')
    qrcode = f.read()
    f.close()
    try:
        cursor.execute("INSERT INTO membersInfo SET userID=%s, name=%s, qrcode=%s", 
                       (str(user_profile.user_id), 
                        str(user_profile.display_name), 
                        qrcode))
        conn.commit()
        # 回覆文字消息與圖片消息
        line_bot_api.reply_message(
            event.reply_token,[
                TextSendMessage(text="歡迎光臨NobodyMart無人超市，感謝您成為NobodyMart會員！"),
                TextSendMessage(text="尊敬的 %s 會員您好！"%(user_profile.display_name)),
                TextSendMessage(text="您的USER_ID是 " + user_profile.user_id),
                TextSendMessage(text="您的QR_code，進入超市前請先出示QR_code驗證身份"),
                ImageSendMessage(original_content_url='https://%s/food/%s.jpg' %(server_url, user_profile.user_id),
                                 preview_image_url='https://%s/food/%s.jpg' %(server_url, user_profile.user_id)),
                TextSendMessage(text="更多資訊請點選下列選單"),       
            ]
        )
    except pymysql.IntegrityError:
        print("Error ", user_profile.user_id, " existed.")
        line_bot_api.reply_message(
            event.reply_token,[
                TextSendMessage(text="Hi～ %s ，歡迎光臨NobodyMart無人超市！"%(user_profile.display_name)),
                TextSendMessage(text="您的USER_ID是 " + user_profile.user_id),
                TextSendMessage(text="您的QR_code，進入超市前請先出示QR_code驗證身份"),
                ImageSendMessage(original_content_url='https://%s/food/%s.jpg' %(server_url, user_profile.user_id),
                                 preview_image_url='https://%s/food/%s.jpg' %(server_url, user_profile.user_id)),
                TextSendMessage(text="更多資訊請點選下列NobodyMartMenu"),       
            ]
        )
    except:
        print("Error ", user_profile.user_id)
        line_bot_api.reply_message(
            event.reply_token,[
                TextSendMessage(text="Hi～ %s ，歡迎光臨NobodyMart無人超市！"%(user_profile.display_name)),
                TextSendMessage(text="您的USER_ID是 " + user_profile.user_id),
                TextSendMessage(text="您的QR_code，進入超市前請先出示QR_code驗證身份"),
                ImageSendMessage(original_content_url='https://%s/food/%s.jpg' %(server_url, user_profile.user_id),
                                 preview_image_url='https://%s/food/%s.jpg' %(server_url, user_profile.user_id)),
                TextSendMessage(text="更多資訊請點選下列NobodyMartMenu"),       
            ]
        )
    #print('userID existed.')
    


In [5]:
'''

建立圖片消息素材

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    ImageSendMessage, TextSendMessage
)

# 圖片消息的基本建構特徵
image_message = TextSendMessage(text='2019-01-23\n雞肉(1)($ 80)\n麵粉(1)($ 50)\n------------------\nTotal  $130')
image_message2 = TextSendMessage(text='暫無紀錄')
image_message3 = TextSendMessage(text='此功能未實裝')

image_message4 = TextSendMessage(text='不能改哦～這功能放好玩的！')

In [6]:
'''

設計一個Carousel Flex
引入carousel的python檔
引入交易紀錄查詢的python程式

'''
# 食譜推薦
import recipe_recommanded_carousel

# 會員中心
import member_center

# 食譜推薦
flexCarouselContainerJsonDict = recipe_recommanded_carousel.flexCarouselContainerJsonDict

# 會員中心

# 交易紀錄
import transaction_history

In [7]:
'''

將carousel類型的json 進行轉換變成 Python可理解之類型物件

將該物件封裝進 Flex Message中


'''

from linebot.models import(
    FlexSendMessage,CarouselContainer,BubbleContainer
)

import json

carouselContent = CarouselContainer.new_from_json_dict(json.loads(flexCarouselContainerJsonDict))
flexCarouselSendMeesage =  FlexSendMessage(alt_text="hello", contents=carouselContent)

In [8]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
    "[::text:]本次交易":image_message,
    "[::text:]歷史交易":image_message2,
    "[::flex:]促銷食材":flexCarouselSendMeesage,
    "[::text:]會員中心":image_message3,
    "[::text:]more":image_message3,
    "[::text:]Modify":image_message4,
    "[::text:]tryLiff":image_message3
}

# Linebot  sendReplyMessage

In [9]:
'''

當用戶發出文字消息時，判斷文字內容是否包含[::text:]，
    若有，則從template_message_dict 內找出相關訊息
    若無，則回傳預設訊息。

'''
from random import randint

# 用來將過多的Liff刪除
from liff_api import *
# initiate token
liff_init(secretFileContentJson.get("channel_access_token"))

# 使用Line notify將使用者傳來的訊息導向管理員的回覆畫面
import line_notify

import imp
import sys
import os

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    # to remove leftover liff
    try:
        if len(liff_list()['apps']) >= 15:
            for i in liff_list()['apps']:
                liff_delete(i['liffId'])
    except:
        print('Liffs have been all removed.')
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
    notify_token = secretFileContentJson.get("notify_token")
    notify_msg = """
    From : %s
    UserName : %s
    Said : %s
    -----------
    Reply
    https://%s/pushMessage/%s
    """%(user_profile.user_id, user_profile.display_name, event.message.text, server_url, user_profile.user_id)
    print(notify_msg)
    line_notify.lineNotify(notify_token, notify_msg)
    
    # 更改為該會員資訊
    if(event.message.text.find('[::text:]會員中心')!= -1):
        qrcodeurl = 'https://%s/food/%s.jpg' %(server_url, user_profile.user_id)
        cursor.execute("SELECT name FROM membersInfo WHERE userID='%s'"%(user_profile.user_id))

        try:
            username = (cursor.fetchone()['name']).encode('utf-8').decode('unicode-escape')
        except TypeError:
            print('TypeError in username.')
            username = 'User name error please modify'
        except:
            print('Unexpected error.')
            username = 'User name error please modify'
        flexCarouselContainerJsonDict_member_center = member_center.member_centre_flex(user_profile.user_id, server_url, qrcodeurl, (user_profile.user_id), username)    
        try:
            carouselContent_member_center = CarouselContainer.new_from_json_dict(json.loads(flexCarouselContainerJsonDict_member_center))
        except json.decoder.JSONDecodeError:
            print('json.decoder.JSONDecodeError in handle_message.')
            username = 'User name error please modify'
            flexCarouselContainerJsonDict_member_center = member_center.member_centre_flex(user_profile.user_id, server_url, qrcodeurl, (user_profile.user_id), username)
            carouselContent_member_center = CarouselContainer.new_from_json_dict(json.loads(flexCarouselContainerJsonDict_member_center))
        flexCarouselSendMeesage_member_center =  FlexSendMessage(alt_text="Member centre", contents=carouselContent_member_center)

        template_message_dict["[::text:]會員中心"] = flexCarouselSendMeesage_member_center
        
    # Try Liff
    if(event.message.text.find('[::text:]tryLiff')!= -1):
        flexBubbleContainerJsonString_transaction_data = transaction_history.tryLiff()
        bubbleContainer_transaction_data= BubbleContainer.new_from_json_dict(json.loads(flexBubbleContainerJsonString_transaction_data))
        flexBubbleSendMessage_transaction_data =  FlexSendMessage(alt_text="try", contents=bubbleContainer_transaction_data)
        template_message_dict["[::text:]tryLiff"] = flexBubbleSendMessage_transaction_data
    
    # 透過MySQL查詢歷史交易
    if(event.message.text.find('[::text:]歷史交易')!= -1):
        conn.commit()
        cursor.execute("SELECT datetime FROM transaction WHERE userID = '%s' GROUP BY orderID DESC LIMIT 7"%(user_profile.user_id))
        transaction_data = cursor.fetchall()
        # 將交易紀錄資料送入flex轉換的transaction_history.py python程式
        flexBubbleContainerJsonString_transaction_data = transaction_history.transaction_flex(user_profile.user_id, server_url, transaction_data)
        bubbleContainer_transaction_data= BubbleContainer.new_from_json_dict(json.loads(flexBubbleContainerJsonString_transaction_data))
        flexBubbleSendMessage_transaction_data =  FlexSendMessage(alt_text="Transaction history", contents=bubbleContainer_transaction_data)
        template_message_dict["[::text:]歷史交易"] = flexBubbleSendMessage_transaction_data
    
    # 本次即時交易紀錄
    if(event.message.text.find('[::text:]本次交易')!= -1):
        # 將UserID送入flex轉換的transaction_history.py python程式
        flexBubbleContainerJsonString_transaction_data = transaction_history.transaction_now_flex(user_profile.user_id, server_url)
        bubbleContainer_transaction_data= BubbleContainer.new_from_json_dict(json.loads(flexBubbleContainerJsonString_transaction_data))
        flexBubbleSendMessage_transaction_data =  FlexSendMessage(alt_text="Current transaction", contents=bubbleContainer_transaction_data)
        template_message_dict["[::text:]本次交易"] = flexBubbleSendMessage_transaction_data
    
    # 食譜推薦，調用暫時carousel
    if(event.message.text.find('[::flex:]促銷食材')!= -1):
        # 導入暫時carousel的py檔
        tmp_user_id = user_profile.user_id
        if os.path.isfile(r'./%s.py'%(tmp_user_id)):
            carousel_module = '%s'%(tmp_user_id)
            mod = __import__(carousel_module, fromlist=[''])        
            print('mod',mod)
            dirmod = dir(mod)
            print('dirmod',dirmod)
            #flexCarouselContainerJsonDict      
            imp.reload(sys.modules['%s'%(tmp_user_id)]) 
            flexCarouselContainerJsonDict = sys.modules['%s'%(tmp_user_id)].flexCarouselContainerJsonDict
            carouselContent = CarouselContainer.new_from_json_dict(json.loads(flexCarouselContainerJsonDict))
            flexCarouselSendMeesage =  FlexSendMessage(alt_text="recommanded", contents=carouselContent)
            template_message_dict["[::flex:]促銷食材"] = flexCarouselSendMeesage
        else:
            import U7173dc299af6ac3fe7697a75c2a280ad
            #flexCarouselContainerJsonDict      
            imp.reload(U7173dc299af6ac3fe7697a75c2a280ad) 
            flexCarouselContainerJsonDict = U7173dc299af6ac3fe7697a75c2a280ad.flexCarouselContainerJsonDict
            carouselContent = CarouselContainer.new_from_json_dict(json.loads(flexCarouselContainerJsonDict))
            flexCarouselSendMeesage =  FlexSendMessage(alt_text="recommanded", contents=carouselContent)
            template_message_dict["[::flex:]促銷食材"] = flexCarouselSendMeesage
    
    if(event.message.text.find('::text:')!= -1) or (event.message.text.find('::flex:')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    elif(event.message.text.find('白痴')!= -1) or (event.message.text.find('笨蛋')!= -1) or (event.message.text.find('智障')!= -1) or (event.message.text.find('神經病')!= -1) or (event.message.text.find('有病')!= -1) or (event.message.text.find('王八蛋')!= -1) or (event.message.text.find('傻蛋')!= -1) or (event.message.text.find('台灣人')!= -1) :
        tmpstr = ''
        if event.message.text.find('笨蛋')!= -1:
            tmpstr = '笨蛋'
        if event.message.text.find('白痴')!= -1:
            tmpstr = '白痴'
        if event.message.text.find('智障')!= -1:
            tmpstr = '智障'
        if event.message.text.find('神經病')!= -1:
            tmpstr = '神經病'
        if event.message.text.find('有病')!= -1:
            tmpstr = '有病'
        if event.message.text.find('王八蛋')!= -1:
            tmpstr = '王八蛋'
        if event.message.text.find('傻蛋')!= -1:
            tmpstr = '傻蛋'
        if event.message.text.find('台灣人')!= -1:
            tmpstr = '台灣人'
        if randint(0,4) == 1:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='你才%s！'%(tmpstr))
            )
        elif randint(0,4) == 2:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='%s才會說別人是%s！'%(tmpstr, tmpstr))
            )
        elif randint(0,4) == 3:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='今天天氣真好！')
            )
        elif randint(0,4) == 4:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='你媽貴姓？！')
            )
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='你全家都%s！'%(tmpstr))
            )
    elif (event.message.text.find('幹你娘')!= -1) or (event.message.text.find('靠北')!= -1) or (event.message.text.find('靠背')!= -1) or (event.message.text.find('操你')!= -1) or (event.message.text.find('操他媽')!= -1) or (event.message.text.find('靠被')!= -1) or (event.message.text.find('幹他媽')!= -1) or (event.message.text.find('幹你')!= -1) or (event.message.text.find('靠杯')!= -1) or (event.message.text.find('去死')!= -1) or (event.message.text.find('fuck')!= -1) or (event.message.text.find('shit')!= -1) or (event.message.text.find('愛你')!= -1) or (event.message.text.find('按陰陽')!= -1) or (event.message.text.find('喜歡你')!= -1) or (event.message.text.find('討厭')!= -1):
        if randint(0,4) == 1:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='你確定？')
            )
        elif randint(0,4) == 2:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='這樣不太好吧？')
            )
        elif randint(0,4) == 3:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='今天天氣真好！')
            )
        elif randint(0,4) == 4:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='你媽貴姓？！')
            )
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='讓我考慮考慮')
            )
    else:
        '''line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=["請點擊功能選單，或輸入[::text:]more，取得更多幫助"+str(event.message.text)+"test", "second"])
        )'''
        line_bot_api.reply_message(
        event.reply_token,
        {'%s'%(str(event.message.text)) : [TextSendMessage(text='%s'%(str(event.message.text).replace('?', '!').replace('？', '！').replace('嗎','').replace('我','你'))), TextSendMessage(text='點擊下方NobodyMartMenu，取得更多幫助。')],
        '更多幫助':image_message3,
        '更多資訊':image_message3,
        'more':image_message3}.get(event.message.text)
        )
        '''{"%s"%(str(event.message.text)) : ['123', 'test', '456']}.get(event.message.text)'''

# 歷史交易紀錄

In [10]:
'''

查詢歷史交易紀錄之接口

'''

@app.route('/transaction/<userID>/<datetime>', methods = ['GET'])
def transactionInfo(userID, datetime):
    # transaction query
    cursor.execute("SELECT tr.orderID, pt.productName, tr.productQTY, pt.productUnitPrice, tr.productQTY*pt.productUnitPrice subtotal FROM transaction tr \
    LEFT JOIN productTable pt \
    ON tr.productID = pt.productID \
    WHERE userID = '%s' AND datetime='%s'"%(userID, datetime))
    trInfo = cursor.fetchall()
    # total
    cursor.execute("SELECT SUM(tr.productQTY*pt.productUnitPrice) total FROM transaction tr \
    LEFT JOIN productTable pt \
    ON tr.productID = pt.productID \
    WHERE userID = '%s' AND datetime='%s'"%(userID, datetime))
    trTotal = cursor.fetchall()
    
    # head
    outStr = """
    <html>
    <head>
    <title>Order_ID : %s</title>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
    <link href="https://%s/food/web_js_css/flexslider.css" rel="stylesheet" type="text/css" />
    <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    
    </head>
    """%(trInfo[0]['orderID'], server_url)
    # body
    outStr += """
    <body>  
    
    <script src="https://%s/food/web_js_css/jquery.min.js"></script>
    <script src="https://%s/food/web_js_css/jquery.flexslider.js"></script>
    <div class="w3-animate-opacity">
    <div class="col-lg-9 col-lg-offset-9">
    """%(server_url, server_url)
    # body-table
    outStr += """
    <table class="table" style="font-size:30px;font-family:serif;">
                <thead>
                    <tr class="danger">
                        <th>No.</th>
                        <th>Product Name</th>
                        <th>Quantity</th>
                        <th>Unit Price</th>
                        <th>Subtotal</th>
                    </tr>
                </thead>    
    """
    # tbody
    outStr += """
    <tbody>
    """
    for i in range(0, len(trInfo)):
        tcolor = ["success", "warning"]
        outStr += """
        <tr class="%s">
            <td>%s</td>
            <td>%s</td>
            <td>%s</td>
            <td>$ %s</td>
            <td>$ %s</td>
        </tr>
        """%(tcolor[i%2], i+1, trInfo[i]['productName'], trInfo[i]['productQTY'], trInfo[i]['productUnitPrice'], trInfo[i]['subtotal'])
    outStr += """
    <thead>
        <tr class="danger">
        <th>Total</th>
        <th>-</th>
        <th>-</th>
        <th>-</th>
        <th>$ %s</th>
        </tr>
    </thead>
    """%(trTotal[0]['total'])
    outStr += """
    </tbody>
    </table>
    """
    # carousel
    outStr += """
    <br><br>
    <div style="margin:0 auto;"><h1><p align=center>-  On-sale products  -</p><h1></div>
    <div class="container">
    <script type="text/javascript">
    $(window).load(function() {
    $('.flexslider').flexslider({
          slideshowSpeed: 5000,
          animationSpeed: 500,
          animation: "slide",
          touch: true
          });
    });
    </script>
    <div class="flexslider" style="width:450px;  margin:0 auto;">
    <ul class="slides">
        <li><img src="https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/2018/05/29/draft/4804805.jpeg&x=0&y=0&sw=0&sh=0&exp=3600" alt="" width="350" height="200"></li>
        <li><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQgszxK4wWxfhfCokY5PwlL52bzi6FTa7vILrrewvOPW6-jDeYGzg" alt="" width="350" height="200"></li>
        <li><img src="https://www.foodnext.net/dispPageBox/getFile/GetImg.aspx?FileLocation=/PJ-FOODNEXT/Files/&FileName=photo-08083-i.jpg" alt="" width="350" height="200"></li>
    </ul>
    </div>
    </div>
    """

    # logo image
    outStr += """
    <div class="container">
    <img class="img-responsive" src="https://%s/food/logo.png">
    </div>
    """%(server_url)
    # /body
    outStr += """
    </div>
    </div>
    </body>
    </html>
    """
    
    return outStr


# 即時交易紀錄(購物車)

In [11]:
'''

查詢當下交易紀錄之接口

'''

@app.route('/transaction/now/<userID>', methods = ['GET'])
def transactionNow(userID):
    # head
    outStr = """
    <html>
    <head>
    <title>User_ID : %s</title>
    <meta http-equiv="refresh" content="5" />
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
    <link href="https://%s/food/web_js_css/flexslider.css" rel="stylesheet" type="text/css" />
    <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    </head>
    """%(userID, server_url)
    # body
    outStr += """
    <body>
    <script src="https://%s/food/web_js_css/jquery.min.js"></script>
    <script src="https://%s/food/web_js_css/jquery.flexslider.js"></script>
    <div class="w3-animate-opacity">
    <div class="col-lg-9 col-loutStrrrset-9">
    """%(server_url, server_url)
    # body-table
    outStr += """
    <table class="table" style="font-size:30px;font-family:serif;">
                <thead>
                    <tr class="danger">
                        <th>No.</th>
                        <th>Product Name</th>
                        <th>Quantity</th>
                        <th>Unit Price</th>
                        <th>Subtotal</th>
                    </tr>
                </thead>    
    """
    # tbody
    outStr += """
    <tbody>
    """
    # 及時交易紀錄
    tcolor = ["success", "warning"]
    '''outStr += """
    <tr class="success">
        <td>1</td>
        <td>Salt</td>
        <td>1</td>
        <td>$ 40</td>
        <td>$ 40</td>
    </tr>
    <tr class="warning">
        <td>2</td>
        <td>Egg</td>
        <td>3</td>
        <td>$ 30</td>
        <td>$ 90</td>
    </tr>
    <tr class="success">
        <td>3</td>
        <td>Chicken</td>
        <td>1</td>
        <td>$ 150</td>
        <td>$ 150</td>
    </tr>
    <tr class="warning">
        <td>4</td>
        <td>OliveOil</td>
        <td>1</td>
        <td>$ 120</td>
        <td>$ 120</td>
    </tr>
    """'''
    with open('./transaction_status/%s'%(userID), 'r', encoding='utf-8') as f:
        outStr += f.read()
    
    '''outStr += """
    <thead>
        <tr class="danger">
        <th>Total</th>
        <th>-</th>
        <th>-</th>
        <th>-</th>
        <th>$ null</th>
        </tr>
    </thead>
    """'''
    
    # /tbody /table
    outStr += """
    </tbody>
    </table>
    """
    
    # carousel
    outStr += """
    <br><br>
    <div style="margin:0 auto;"><h1><p align=center>-  On-sale products  -</p><h1></div>
    <div class="container">
    <script type="text/javascript">
    $(window).load(function() {
    $('.flexslider').flexslider({
          slideshowSpeed: 5000,
          animationSpeed: 500,
          animation: "slide",
          touch: true
          });
    });
    </script>
    <div class="flexslider" style="width:450px;  margin:0 auto;">
    <ul class="slides">
        <li><img src="https://pgw.udn.com.tw/gw/photo.php?u=https://uc.udn.com.tw/photo/2018/05/29/draft/4804805.jpeg&x=0&y=0&sw=0&sh=0&exp=3600" alt="" width="350" height="200"></li>
        <li><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQgszxK4wWxfhfCokY5PwlL52bzi6FTa7vILrrewvOPW6-jDeYGzg" alt="" width="350" height="200"></li>
        <li><img src="https://www.foodnext.net/dispPageBox/getFile/GetImg.aspx?FileLocation=/PJ-FOODNEXT/Files/&FileName=photo-08083-i.jpg" alt="" width="350" height="200"></li>
    </ul>
    </div>
    </div>
    """
    
    # logo image
    outStr += """
    <div class="container">
    <img class="img-responsive" src="https://%s/food/logo.png">
    </div>
    """%(server_url)
    # /body
    outStr += """
    </div>
    </div>
    </body>
    """
    
    # test
    '''outStr = """"""
    with open('./transaction_status/%s'%(userID), 'r', encoding='utf-8') as f:
        outStr += f.read()'''
    
    return outStr

# 會員中心

In [12]:
'''

會員中心資料修改

'''

@app.route('/usercentre/<userID>', methods=['GET'])
def getdata(userID):
    outStr = """
    <!doctype html>
    <html>
    <head>   
    
    <title>USER CENTRE</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    
    </head>
    
    <body>
    <script type="text/javascript">
    function ValidateValue(textbox) {
        var IllegalString = "[`~!#$^&*()=|{}':;',\\[\\].<>/?~！#￥……&*（）——|{}【】‘；：”“'。，、？]‘'";
        var textboxvalue = textbox.value;
        var index = textboxvalue.length - 1;
        var s = textbox.value.charAt(index);
        if (IllegalString.indexOf(s) >= 0) {
        s = textboxvalue.substring(0, index);
        textbox.value = s;
        }
    }
    </script>
    <div class="w3-animate-opacity">
        <table class="table" style="font-family:serif;margin:0 auto;">
        <thead>
        <th>
        Hello, your name is ?
        </th>
        </thead>
        </table>
    <p></p>
    <table class="table" style="font-family:serif;margin:0 auto;">
    <thead>
    <th>
        <form action="/usercentre/%s" method="post">
         <label>Name:</label>
         <input type="textbox" name="username" onkeyup = "ValidateValue(this)">
         <input type="submit" value="Submit">
        </form>
    </th>
    </thead>
    </table>
    </div>
    </body>
    </html>
    """%(userID)
    #return render_template('text.html')
    return outStr

@app.route('/usercentre/<userID>', methods=['POST'])
def submit(userID):
    name = request.form.get('username')
    print(name)
    print('Find \\ ', ('\\' in name), ' , in submit')
    
    content = """
    Please try a shorter name.
    <br>
    <input type="button" value="Return" onclick="location.href='https://%s/usercentre/%s'">
    """%(server_url, userID)
    
    if ('\\' in name) or (len(name) == 0):
        name = "ErrorTyping"
        content = """
        Please type regular words (\).
        <br>
        <input type="button" value="Return" onclick="location.href='https://%s/usercentre/%s'">
        """%(server_url, userID)
    elif len(name.encode('unicode-escape').decode('utf-8').replace('\\','\\\\')) < 39 :
        content = """Your user name has been modified to <br>%s !"""%(name)
        cursor.execute("UPDATE membersInfo SET name='%s' WHERE userID='%s'"%(name.encode('unicode-escape').decode('utf-8').replace('\\','\\\\'), userID))
        conn.commit()    
    
    outStr = """
    <html>
    <head>   
    
    <title>USER CENTRE</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    
    </head>
    
    <body>
    
    <table class="table" style="font-family:serif;margin:0 auto;">
    <thead>
    <th>
    %s
    </th>
    </thead>
    </table>
    
    </body>
    </html>
    """%(content)
    return outStr

# Push message
## 管理員回復訊息
## 交易結束推播訊息

In [13]:
'''

Push message
Reply message manually

'''
from linebot import LineBotApi
from linebot.models import TextSendMessage
from linebot.exceptions import LineBotApiError

@app.route('/pushMessage/<userID>/<sendText>', methods=['GET'])
def pushMessageDemo(userID, sendText):
    try:
        line_bot_api.push_message(userID, TextSendMessage(text='%s'%(sendText)))
    except LineBotApiError as e:
        # error handle
        print('Push Error.')
    
    return 'Done.'

# reply
@app.route('/pushMessage/<userID>', methods=['GET'])
def replyMessageManually(userID):
    
    outStr = """
    <!doctype html>
    <html>
    <head>   
    
    <title>REPLY</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    
    </head>
    
    <body>
    
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
        <thead>
        <th>
        Say something
        </th>
        </thead>
        </table>
    <p></p>
    <table class="table" style="font-family:serif;margin:0 auto;">
    <thead>
    <th>
        <form action="/pushMessage/%s" method="post">
         <label>Reply:</label>
         <input type="textbox" name="replyMessage">
         <input type="submit" value="Submit">
        </form>
    </th>
    </thead>
    </table>
    </div>
    </body>
    </html>
    """%(userID)
    
    return outStr
@app.route('/pushMessage/<userID>', methods=['POST'])
def sendReplyMessage(userID):
    replyMessage = request.form.get('replyMessage')
    replyStatus = "Sent."
    try:
        line_bot_api.push_message(userID, TextSendMessage(text='%s'%(replyMessage)))
    except LineBotApiError as e:
        # error handle
        replyStatus = "Error!"
        print('Push Error.')
    outStr = """
    <!doctype html>
    <html>
    <head>   
    
    <title>REPLY</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    
    </head>
    
    <body>
    
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
        <thead>
        <th>
        %s
        </th>
        </thead>
        </table>
        <p></p>
        <table class="table" style="font-family:serif;margin:0 auto;">
        <thead>
        <th>
            <form action="/pushMessage/%s" method="post">
             <label>Reply:</label>
             <input type="textbox" name="replyMessage">
             <input type="submit" value="Submit">
            </form>
        </th>
        </thead>
        </table>
    </div>
    </body>
    </html>
    """%(replyStatus, userID)
    
    return outStr

@app.route('/transaction_commit/<userID>')
def commit_data_and_push(userID):
    conn.commit()
    #conn.reconnect()
    
    cursor.execute("SELECT datetime FROM transaction WHERE userID = '%s' GROUP BY orderID DESC LIMIT 1"%(userID))
    transaction_data = cursor.fetchall()
    # 將交易紀錄資料送入flex轉換的transaction_history.py python程式
    flexBubbleContainerJsonString_transaction_data = transaction_history.transaction_flex(userID, server_url, transaction_data)
    bubbleContainer_transaction_data= BubbleContainer.new_from_json_dict(json.loads(flexBubbleContainerJsonString_transaction_data))
    FlexSendMessage(alt_text="Transaction history", contents=bubbleContainer_transaction_data)
    
    #replyStatus = "Sent."
    try:
        line_bot_api.push_message(userID, [FlexSendMessage(alt_text="Transaction history", contents=bubbleContainer_transaction_data),
                                          TextSendMessage(text='感謝您的光臨，您可於上方表單查看最新交易紀錄。')])
        print('From : %s\nTransaction Completed!'%(userID))
    except LineBotApiError as e:
        # error handle
        #replyStatus = "Error!"
        print('Push Error.')
    
    return 'Done.'

# Kafka專用

In [14]:
'''

From face PI
Push message
Reply message manually

'''
from linebot import LineBotApi
from linebot.models import TextSendMessage, ImageSendMessage
from linebot.exceptions import LineBotApiError


@app.route('/tryagain/<userID>', methods=['GET'])
def pushTryagain(userID):
    try:
        line_bot_api.push_message(userID, TextSendMessage(text='人臉辨識錯誤！請重新掃描QRcode！'))
    except LineBotApiError as e:
        # error handle
        print('Push Error.')
    
    return 'Done.'

@app.route('/welcom/<userID>', methods=['GET'])
def pushWelcom(userID):
    try:
        line_bot_api.push_message(userID, 
                                  [
                                      TextSendMessage(text='歡迎光臨NobodyMart！您可以點擊下方[NobodyMartMenu] -> [本次交易] 查看本次交易的購物車。'),
                                      ImageSendMessage(original_content_url='https://%s/food/transaction_now.png' %(server_url),
                                         preview_image_url='https://%s/food/transaction_now.png' %(server_url))
                                  ]
                                 )
    except LineBotApiError as e:
        # error handle
        print('Push Error.')
    
    # Write userID to a textfile
    outStr = """
    <html>
        <head>
            <title>%s</title>
        </head>
        <body>
            <div>%s</div>
        </body>
    </html>
    """%(userID, userID)
    with open('./templates/userNow', 'w', encoding='utf-8') as h:
        h.write(outStr)
    
    return 'Done.'

@app.route('/welcom', methods=['GET'])
def topicName():
    #from flask import render_template
    #return render_template('userNow.html')
    
    outStr = """"""
    
    with open('./templates/userNow', 'r', encoding='utf-8') as h:
        outStr = h.read()
        
    return outStr

@app.route('/qrcode/<userID>', methods=['GET'])
def pushQRcodeWelcom(userID):
    cursor.execute("SELECT name FROM membersInfo WHERE userID='%s'"%(userID))
    username = cursor.fetchone()['name'].encode('utf-8').decode('unicode-escape')
    try:
        line_bot_api.push_message(userID, TextSendMessage(text='Hi～%s，歡迎光臨NobodyMart，請看著鏡頭做臉部辨識，以進行雙重確認。'%(username)))
    except LineBotApiError as e:
        # error handle
        print('Push Error.')
    return 'Done'

# 管理員專區

In [15]:
'''

Administrator
商品庫存管理
會員資料查詢
會員交易紀錄查詢
客流量分析
商品熱度圖

'''

@app.route('/administrator', methods=['GET'])
def administrator_only():
    outStr = """
    <!doctype html>
    <html>
    <head>
        <title>ADMINISTRATOR</title>
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <!--meta name="viewpoutStr" content="width=device-width, initial-scale=1.0"-->
        <!--meta http-equiv="refresh" content="0;url=http://%s/passengerFlows"/-->
        <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    </head>
    
    <body>
    <div class="w3-animate-opacity">
    """%(server_url)
    
    outStr += """
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <h1>UNDER CONSTRUCTION!</h1>
                </th>
            </thead>
        </table>
        <br>
        
    </div>
    """
    
    # 商品庫存管理
    outStr += """
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <input type="button" value="商品庫存管理" onclick="location.href='http://%s/productStorage'">
                </th>
            </thead>
        </table>
        <br>       
    </div>    
    """%(server_url)
    
    # 會員資料查詢
    outStr += """
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <input type="button" value="會員資料查詢" onclick="location.href='http://%s/memberInformation'">
                </th>
            </thead>
        </table>
        <br>       
    </div>     
    """%(server_url)
    
    # 會員交易紀錄查詢
    outStr += """
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <input type="button" value="會員交易紀錄查詢" onclick="location.href='http://%s/memberTransaction'">
                </th>
            </thead>
        </table>
        <br>       
    </div> 
    """%(server_url)
    
    # 客流量分析
    outStr += """
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <input type="button" value="客流量分析" onclick="location.href='http://%s/passengerFlows'">
                </th>
            </thead>
        </table>
        <br>       
    </div> 
    """%(server_url)
    
    # 商品熱度圖
    outStr += """
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <input type="button" value="商品熱度圖" onclick="location.href='http://%s/productHeatMap'">
                </th>
            </thead>
        </table>
        <br>       
    </div> 
    """%(server_url)
    
    outStr += """
    </div>
    </body>
    """
    
    outStr += """
    </html>
    """
    
    return outStr

### 商品庫存管理
@app.route('/productStorage', methods=['GET'])
def productStorage():
    outStr = """
    <!doctype html>
    <html>
    <head>
        <title>UNDER CONSTRUCTION</title>
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    </head>
    
    <body>
    <div class="w3-animate-opacity">
    """
    
    outStr += """
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <h1>UNDER CONSTRUCTION!</h1>
                </th>
            </thead>
        </table>
        
        <br>
        
        
        
        <br>
        
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <input type="button" value="Return" onclick="location.href='https://%s/administrator'">
                </th>
            </thead>
        </table>
    </div>
    """%(server_url)
    
    outStr += """
    </div>
    </body>
    """
    
    outStr += """
    </html>
    """
    
    return outStr

### 會員資料查詢
@app.route('/memberInformation', methods=['GET'])
def memberInformation():
    outStr = """
    <!doctype html>
    <html>
    <head>
        <title>UNDER CONSTRUCTION</title>
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    </head>
    
    <body>
    <div class="w3-animate-opacity">
    """
    
    outStr += """
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <h1>UNDER CONSTRUCTION!</h1>
                </th>
            </thead>
        </table>
        
        <br>
        
        
        
        <br>
        
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <input type="button" value="Return" onclick="location.href='https://%s/administrator'">
                </th>
            </thead>
        </table>
    </div>
    """%(server_url)
    
    outStr += """
    </div>
    </body>
    """
    
    outStr += """
    </html>
    """
    
    return outStr

### 會員交易紀錄查詢
@app.route('/memberTransaction', methods=['GET'])
def memberTransaction():
    outStr = """
    <!doctype html>
    <html>
    <head>
        <title>UNDER CONSTRUCTION</title>
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    </head>
    
    <body>
    <div class="w3-animate-opacity">
    """
    
    outStr += """
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <h1>UNDER CONSTRUCTION!</h1>
                </th>
            </thead>
        </table>
        
        <br>
        
        
        
        <br>
        
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <input type="button" value="Return" onclick="location.href='https://%s/administrator'">
                </th>
            </thead>
        </table>
    </div>
    """%(server_url)
    
    outStr += """
    </div>
    </body>
    """
    
    outStr += """
    </html>
    """
    
    return outStr

### 客流量分析
@app.route('/passengerFlows', methods=['GET'])
def passengerFlows():
    outStr = """
    <!doctype html>
    <html>
    <head>
        <title>UNDER CONSTRUCTION</title>
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    </head>
    
    <body>
    <div class="w3-animate-opacity">
    """
    
    outStr += """
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <h1>Passengre Flows</h1>
                </th>
            </thead>
        </table>
        <br>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <iframe src="http://35.220.196.201:3000/d-solo/xjasT46iz/today?orgId=1&panelId=2" width="350" height="200" frameborder="0"></iframe>
                </th>
            </thead>
            <thead>
                <th>
                    <iframe src="http://35.220.196.201:3000/d-solo/aklDtA3ik/nobodymart?orgId=1&panelId=2" width="350" height="200" frameborder="0"></iframe>
                </th>
            </thead>
            <thead>
                <th>
                    <iframe src="http://35.220.196.201:3000/d-solo/bCSqTV6mk/7days?orgId=1&panelId=2" width="350" height="200" frameborder="0"></iframe>
                </th>
            </thead>
        </table>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <input type="button" value="Return" onclick="location.href='https://%s/administrator'">
                </th>
            </thead>
        </table>
    </div>
    """%(server_url)
    
    outStr += """
    </div>
    </body>
    """
    
    outStr += """
    </html>
    """
    
    return outStr

### 商品熱度圖
@app.route('/productHeatMap', methods=['GET'])
def productHeatMap():
    outStr = """
    <!doctype html>
    <html>
    <head>
        <title>UNDER CONSTRUCTION</title>
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    </head>
    
    <body>
    <div class="w3-animate-opacity">
    """
    
    outStr += """
    <div>
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <h1>UNDER CONSTRUCTION!</h1>
                </th>
            </thead>
        </table>
        
        <br>
        
        
        
        <br>
        
        <table class="table" style="font-family:serif;margin:0 auto;">
            <thead>
                <th>
                    <input type="button" value="Return" onclick="location.href='https://%s/administrator'">
                </th>
            </thead>
        </table>
    </div>
    """%(server_url)
    
    outStr += """
    </div>
    </body>
    """
    
    outStr += """
    </html>
    """
    
    return outStr

In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    #app.debug = True
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
192.168.224.4 - - [29/Mar/2019 03:30:46] "GET /administrator HTTP/1.1" 200 -
192.168.224.4 - - [29/Mar/2019 03:30:48] "GET /passengerFlows HTTP/1.1" 200 -
192.168.224.4 - - [29/Mar/2019 03:31:28] "GET /administrator HTTP/1.1" 200 -
192.168.224.4 - - [29/Mar/2019 03:31:30] "GET /passengerFlows HTTP/1.1" 200 -
192.168.224.4 - - [29/Mar/2019 03:31:30] "POST / HTTP/1.1" 200 -
192.168.224.4 - - [29/Mar/2019 03:54:44] "GET /transaction_commit/Ua0ac6b70d3611ea9d3d4665941dbf3f7 HTTP/1.1" 200 -


From : Ua0ac6b70d3611ea9d3d4665941dbf3f7
Transaction Completed!


192.168.224.4 - - [29/Mar/2019 03:54:55] "GET /transaction_commit/U26c1d062b5768c0ab8721698973d67b1 HTTP/1.1" 200 -


From : U26c1d062b5768c0ab8721698973d67b1
Transaction Completed!


192.168.224.4 - - [29/Mar/2019 03:55:07] "GET /transaction_commit/Uc132d7550826d16f3d86f60a24aaae20 HTTP/1.1" 200 -


...

[2019-03-30 06:58:39,415] ERROR in app: Exception on /transaction_commit/U26c1d062b5768c0ab8721698973d67b1 [GET]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-13-1b9091f3fa5c>", line 120, in commit_

192.168.224.4 - - [30/Mar/2019 06:59:34] "GET /transaction_commit/Uc132d7550826d16f3d86f60a24aaae20 HTTP/1.1" 500 -
[2019-03-30 06:59:45,584] ERROR in app: Exception on /transaction_commit/Udf8f1c512878156d2bf8d0051d4f2429 [GET]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    retur

192.168.224.4 - - [30/Mar/2019 07:00:40] "GET /transaction_commit/U934211376c8da4cc677c3b801bd9f8db HTTP/1.1" 500 -
[2019-03-30 07:00:51,879] ERROR in app: Exception on /transaction_commit/U1e91bf35eb5315bc17da624e46a65c51 [GET]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    retur

192.168.224.4 - - [30/Mar/2019 07:01:46] "GET /transaction_commit/Udf8f1c512878156d2bf8d0051d4f2429 HTTP/1.1" 500 -
[2019-03-30 07:01:57,767] ERROR in app: Exception on /transaction_commit/U1e91bf35eb5315bc17da624e46a65c51 [GET]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    retur

192.168.224.4 - - [30/Mar/2019 07:02:52] "GET /transaction_commit/U9a1715c6ffdf0ef2ffe148726d336011 HTTP/1.1" 500 -
[2019-03-30 07:03:04,002] ERROR in app: Exception on /transaction_commit/Udf8f1c512878156d2bf8d0051d4f2429 [GET]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.7/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.7/site-packages/flask/app.py", line 1598, in dispatch_request
    retur

In [ ]:
#pymysql.err.OperationalError
#pymysql.err.InterfaceError